# Resolving

A [Resolver](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resolving) is used to link terms or a `Resource` to identifiers (URIs) in a knowledge graph thus addressing lexical variations
(merging of synonyms, aliases and acronyms) and disambiguating them. This feature is also referred to as entity linking
specially in the context of Natural Language Processing (NLP) when building knowledge graph from entities extracted from
text documents.

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml", debug=True)

## Imports

In [3]:
from kgforge.core.commons.strategies import ResolvingStrategy
from kgforge.core.resource import Resource

## Discover configured resolvers
With the `forge.resolvers()` method, configured resolvers can be inspected.

In [4]:
forge.resolvers()

Available scopes:
 -  entities :
     - resolver:  DemoResolver
         - targets:  agents
 -  schemaorg :
     - resolver:  EntityLinkerSkLearn
         - targets:  terms
 -  terms :
     - resolver:  DemoResolver
         - targets:  sexontology


A scope is a convenient (and arbitrary) way to name a given Resolver along with a set of sources of data (the `targets`) to resolve against. Resolve a resource for `female` in the 'terms' resolving scope.

## DemoResolver

The DemoResolver resolve a term using str comparision and is looking up in a json file.

### scope

 Resolve the text`female` againt the 'terms' resolving scope.

In [5]:
female = forge.resolve(text="female", scope="terms")

In [6]:
type(female)

kgforge.core.resource.Resource

In [7]:
print(female)

{
    id: http://purl.obolibrary.org/obo/PATO_0000383
    type: Class
    label: female
}


Resolve the text `EPFL` against the 'entities' resolving scope.

In [8]:
epfl = forge.resolve("EPFL", scope="entities")

In [9]:
type(epfl)

kgforge.core.resource.Resource

In [10]:
print(epfl)

{
    id: https://www.grid.ac/institutes/grid.5333.6
    type: Organization
    label: École Polytechnique Fédérale de Lausanne
    acronym: EPFL
}


### target

In [11]:
print(forge.resolve("female", scope="terms", target="sexontology"))

{
    id: http://purl.obolibrary.org/obo/PATO_0000383
    type: Class
    label: female
}


In [12]:
print(forge.resolve("EPFL", scope="entities", target="agents"))

{
    id: https://www.grid.ac/institutes/grid.5333.6
    type: Organization
    label: École Polytechnique Fédérale de Lausanne
    acronym: EPFL
}


### type

In [13]:
print(forge.resolve("female", scope="terms", type="Class"))

{
    id: http://purl.obolibrary.org/obo/PATO_0000383
    type: Class
    label: female
}


In [14]:
print(forge.resolve("EPFL", scope="entities", type="Organization"))

{
    id: https://www.grid.ac/institutes/grid.5333.6
    type: Organization
    label: École Polytechnique Fédérale de Lausanne
    acronym: EPFL
}


## Strategies

Different strategies can be used to rank resolving candidates. 

In the following example, the missing 'e' at the end is intended for the demonstration.

In [15]:
text = "mal"

### best match

The default applied strategy is `strategy=ResolvingStrategy.BEST_MATCH`.

In [16]:
print(forge.resolve(text, scope="terms"))

{
    id: http://purl.obolibrary.org/obo/PATO_0000384
    type: Class
    label: male
}


### exact match

In [17]:
print(forge.resolve(text, scope="terms", strategy=ResolvingStrategy.EXACT_MATCH))

None


### fuzzy match (all matches)

The candidates list is ordered by score.

In [18]:
results = forge.resolve(text, scope="terms", strategy=ResolvingStrategy.ALL_MATCHES, limit=3)

In [19]:
type(results)

list

In [20]:
len(results)

2

In [21]:
type(results[0])

kgforge.core.resource.Resource

In [22]:
print(*results, sep="\n")

{
    id: http://purl.obolibrary.org/obo/PATO_0000384
    type: Class
    label: male
}
{
    id: http://purl.obolibrary.org/obo/PATO_0000383
    type: Class
    label: female
}


## Resolving a Resource
A kgforge.core.resource.Resource can be resolved. In such case and in addition to the other supported arguments, the resource property to resolve can be provided through the argument 'property_to_resolve'. The resolving result can be merge back in the input resource by setting the 'merge_inplace_as argument'. When 'merge_inplace_as' is not set then the results are returned as separate resources.

In [23]:
resource = Resource(type="Agent", gender="mal")
print(resource)

{
    type: Agent
    gender: mal
}


In [24]:
resource_resolved = forge.resolve(resource, scope="terms", target="sexontology",
                                  strategy=ResolvingStrategy.ALL_MATCHES,
                                  property_to_resolve="gender",
                                  merge_inplace_as="gender_resolved",
                                  threshold=0.8)

In [25]:
type(resource_resolved)

kgforge.core.resource.Resource

In [26]:
print(resource_resolved)

{
    type: Agent
    gender: mal
    gender_resolved:
    [
        {
            id: http://purl.obolibrary.org/obo/PATO_0000384
            type: Class
            label: male
        }
        {
            id: http://purl.obolibrary.org/obo/PATO_0000383
            type: Class
            label: female
        }
    ]
}


In [27]:
resource_resolved = forge.resolve(resource, scope="terms", target="sexontology",
                                  strategy=ResolvingStrategy.ALL_MATCHES,
                                  property_to_resolve="gender",
                                  threshold=0.8)

In [28]:
type(resource_resolved)

list

In [29]:
len(resource_resolved)

2

In [30]:
print(*resource_resolved, sep="\n")

{
    id: http://purl.obolibrary.org/obo/PATO_0000384
    type: Class
    label: male
}
{
    id: http://purl.obolibrary.org/obo/PATO_0000383
    type: Class
    label: female
}


## EntityLinkerSkLearn Resolver

Based on a pretrained model and using [scikit-learn](https://scikit-learn.org/stable/index.html) to generate and rank candidates.

In [32]:
print(forge.resolve("person", scope="schemaorg", target="terms", strategy=ResolvingStrategy.BEST_MATCH))

{
    id: http://schema.org/Person
    label: Person
    altLabel: Person
    definition: A person (alive, dead, undead, or fictional).
    score: 0.0
}
